# Combine languages with *language code* or language word and convert to json

In [5]:
DATA_PATH = "../../2023InflectionST/part1/data/"
OUT_DIR = "data/"
# LANGS = ["deu","eng"]
# TARGET_LANG = "deu_eng"
# LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"]
# TARGET_LANG = "mul"
LANGS_UNIQUE = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"]+["ote","pol","amh","csb"]
#TARGET_LANG = "mul_2"
LANG_WORD_OF_CODE = {"grc":"Ancient Greek",
                  "dan": "Danish",
                  "fra":"French",
                  "sme":"Sámi",
                  "deu":"German",
                  "nav":"Navajo",
                  "jap":"Japanese",
                  "klr":"Khaling",
                  "eng":"English",
                  "ote":"Mezquital Otomi",
                  "pol":"Polish",
                  "amh":"Amharic",
                  "csb":"Kashubian"}

TARGET_LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","ote","pol","amh","csb","mul"]
DATASET_COMPLETE = ["trn","tst","dev"]
LANGUAGE_CODES = [True,False]
print(OUT_DIR)
for target_lang in TARGET_LANGS:
    target_lang_initials = target_lang
    for lang_code in LANGUAGE_CODES:
        if lang_code: 
            target_lang = target_lang_initials  + "-language-code"
        else: 
            target_lang = target_lang_initials  + "-language-word"
            
        for dataset in DATASET_COMPLETE:
            if dataset != "tst" and  "csb" in target_lang: continue # csb only as test data
                
            OUT=OUT_DIR + target_lang  + "_" + dataset + ".json"
            with open (OUT,"w") as out_file:
                print(out_file)
                if target_lang in ["mul-language-code","mul-language-word"]: LANGS = LANGS_UNIQUE
                else: LANGS = [target_lang_initials]
                for lang in LANGS:
                    if dataset != "tst" and lang == "csb": continue # csb only as test data
                        
                    with open(DATA_PATH + lang + "." + dataset,"r") as in_file:
                        for line in in_file:
                            line = line.strip()
                            lemma, features, target = line.split("\t")
                            if lang_code: context = lang + ": "
                            else: context = LANG_WORD_OF_CODE[lang]+ ": "
                            json_line = f'{{"input": "{context}Inflect {lemma} | {features}","target": "{target}"}}'
                            out_file.write(json_line + "\n")



data/
<_io.TextIOWrapper name='data/grc-language-code_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-code_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-code_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/grc-language-word_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-code_dev.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-word_trn.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-word_tst.json' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='data/dan-language-word_dev.json' m